In [ ]:
# generate dataframe of each note in all songs
pitch = []
rest = []
chrod = []

components = info_midi(path_1 + "C_alb_esp1.mid")
note_list = []

for element in components:
    # note pitches are extracted
    if isinstance(element, note.Note):
        note_list.append(str(element.pitch))
        pitch.append(str(element.pitch))
    # chords are extracted
    elif isinstance(element, chord.Chord):
        note_list.append(".".join(str(n) for n in element.normalOrder))   
        chrod.append(".".join(str(n) for n in element.normalOrder))    
    # rests are extracted
    elif isinstance(element, note.Rest):
        note_list.append("NULL")    #further transformation needs this value as str rather than np.nan
        rest.append("NULL")

a = {"Pitch": pitch, "Rest": rest}#, "Chord": chord}
df = pd. DataFrame.from_dict(a, orient='index')
df = df.transpose()
df

In [ ]:
# get notes from all songs and save them into the same list

def get_all_notes(path=path_1):
    """
    This function extracts all the notes, rests and chords from all midi files 
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file     
    """
    list_path = os.listdir(path)
    for elem in list_path:
        output = get_notes_per_song(path, elem)
    return output

note_list = get_all_notes()

# si tengo la funcion del dataframe necesito esta?''''''''''''''''''''''''''''''''

In [ ]:
def generate_latent_points(note_list, x_real, x):

    pitchnames = sorted(set(note_list))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # create random matrix of ones and zeros 
    noise = np.random.choice(len(pitchnames)-1, (x_real.shape[0], x.shape[1], x.shape[2]))
    x_latent = []
    # transform random int to note
    for elem in noise[0][0]:
        fake = int_to_note[abs(int(elem))]
        x_latent.append(fake)  

    x_latent = np.array(x_latent)

    return x_latent

In [ ]:
def generate_fake_data(note_list, x_real, x, g_model):
	# generate points in latent space
	x_fake = generate_latent_points(note_list, x_real, x)
	# predict outputs
	x_fake = g_model.predict(x_fake)
	# create 'fake' class labels (0)
	y_fake = np.zeros((x.shape[2], 1))
	return x_fake, y_fake